# Import Libraries

In [65]:
import pandas as pd

In [66]:
import torch
import torch.nn as nn
import pandas as pd
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from model import GPTLanguageModel  # Import  model class

# Hyperparameters

In [67]:
# hyperparameters

max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
batch_size = 16 #  independent sequences processed in parallel
block_size = 32 # maximum context length for predictions
# ------------

torch.manual_seed(1337)

In [68]:



def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


# Loading the Dataset

In [69]:

with open('binary_operation_fine_shuffled_file.csv', 'r', encoding='utf-8') as f:
    text = f.read()

In [70]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [71]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]



In [ ]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [55]:
from torch.nn.utils.rnn import pad_sequence

# Train the model

In [11]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 3.8200, val loss 3.8202
step 100: train loss 1.5247, val loss 1.5334
step 200: train loss 1.1749, val loss 1.1842
step 300: train loss 1.1052, val loss 1.1258
step 400: train loss 1.0577, val loss 1.0606
step 500: train loss 1.0185, val loss 1.0185
step 600: train loss 0.9376, val loss 0.9546
step 700: train loss 0.8900, val loss 0.9062
step 800: train loss 0.8407, val loss 0.8733
step 900: train loss 0.8114, val loss 0.8278
step 1000: train loss 0.8061, val loss 0.8199
step 1100: train loss 0.7769, val loss 0.7954
step 1200: train loss 0.7608, val loss 0.7755
step 1300: train loss 0.7429, val loss 0.7706
step 1400: train loss 0.7482, val loss 0.7652
step 1500: train loss 0.7372, val loss 0.7589
step 1600: train loss 0.7238, val loss 0.7496
step 1700: train loss 0.7230, val loss 0.7397
step 1800: train loss 0.7247, val loss 0.7454
step 1900: train loss 0.7001, val loss 0.7264
step 2000: train loss 0.7054, val loss 0.7194
step 2100: train loss 0.6932, val loss 0.7101


In [12]:
import re

def generate_prompt_response(prompt, model, max_new_tokens):
    model.eval()
    input_tokens = encode(prompt)
    context = torch.tensor(input_tokens, dtype=torch.long, device=device).unsqueeze(0)
    generated_tokens = model.generate(context, max_new_tokens=max_new_tokens)[0].tolist()
    generated_response = decode(generated_tokens)

    # Use regular expression to match the answer format (e.g., "3+4=7"or "3x4=12")
    match = re.search(r'\d+\s*([-+\/xX])\s*\d+\s*=\s*\d+', generated_response)
    if match:
        answer = match.group(0)
        return answer

    return "Answer not found"

trained_model = m


In [31]:
prompt = "hey, what's 3+13?"  # The expected result is 16
max_new_tokens = 15  # Maximum number of tokens in the generated response

# Generate response based on the prompt
generated_response = generate_prompt_response(prompt, trained_model, max_new_tokens)
print(f"Input Prompt: {prompt}\nGenerated Response: {generated_response}")

Input Prompt: hey, what's 3+13?
Generated Response: 3+13=15


In [14]:
prompt = "What is 12x10?"  # The expected result is 120
max_new_tokens = 15  # Maximum number of tokens in the generated response

# Generate response based on the prompt
generated_response = generate_prompt_response(prompt, trained_model, max_new_tokens)
print(f"Input Prompt: {prompt}\nGenerated Response: {generated_response}")

Input Prompt: What is 12x10?
Generated Response: 12x10=140


In [25]:
prompt = "What is 12/6?"  # The expected result is 2
max_new_tokens = 15  # Maximum number of tokens in the generated response

# Generate response based on the prompt
generated_response = generate_prompt_response(prompt, trained_model, max_new_tokens)
print(f"Input Prompt: {prompt}\nGenerated Response: {generated_response}")

Input Prompt: What is 12/6?
Generated Response: 12/6=2


In [26]:
prompt = "value of 15+5?"  # The expected result is 20
max_new_tokens = 15  # Maximum number of tokens in the generated response

# Generate response based on the prompt
generated_response = generate_prompt_response(prompt, trained_model, max_new_tokens)
print(f"Input Prompt: {prompt}\nGenerated Response: {generated_response}")

Input Prompt: value of 15+5?
Generated Response: 15+5=22
